In [18]:
from typing import Annotated
from dotenv import load_dotenv #that sets the key from .env as an env variable
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [19]:
llm = init_chat_model("google_genai:gemini-2.5-flash")

In [20]:
llm.invoke("Who was the first person on the moon?").content

'The first person on the moon was **Neil Armstrong**.\n\nHe landed on the moon on July 20, 1969, as part of the Apollo 11 mission. His famous words upon stepping onto the lunar surface were: "That\'s one small step for [a] man, one giant leap for mankind."'

In [24]:
class State(TypedDict): #I'm going to be making messages, so I need list of them
    messages: Annotated[list, add_messages]
    #if it'd be just "mesages: List" it will not automatically preserve the original message
    #add_messages is a reducer_function

def chatbot(state: State) -> State: #a node is a func that will return list of messages based on past messages
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

<class '__main__.State'>


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
graph.invoke({"messages":[message]})

In [ ]:
message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

In [ ]:
state = None
while True: #this can be done better using inbuilt langgraph functions - Memory Saver
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})
    state = graph.invoke(state)
    print("You:", state["messages"][-2].content)
    print("Bot:", state["messages"][-1].content.replace('\n'," "))